# Test the new boundsx and boundsy args

In [ ]:
import numpy as np
import holoviews as hv; hv.extension('bokeh')
from holoviews.plotting.links import RangeToolLink
from neurodatagen.eeg import generate_eeg_powerlaw
from scipy.stats import zscore

In [ ]:
n_channels = 25
n_seconds = 30
fs = 512

data, time, channels = generate_eeg_powerlaw(n_channels, n_seconds, fs)

print(f'shape: {data.shape} (n_channels, samples) ')
data

In [ ]:
spacing = 1.2
offset = np.max(np.abs(data)) * spacing

channel_curves = []
for i, channel_data in enumerate(data):
    channel_curves.append(
        hv.Curve((time, channel_data + (i * offset)), "Time").opts(
            color="black", line_width=1, tools=["hover"]))

yticks = [(i * offset, ich) for i, ich in enumerate(channels)]

# create a hook to set a predefined selection of RangeToolLink
max_ch_disp = 10
max_t_disp = 5
range_opts = []

# def xrange_hook(plot, element):
#     plot.handles['x_range'].end = max_t_disp

# def yrange_hook(plot, element):
#     plot.handles['y_range'].end = np.max(data[max_ch_disp-1,:] + ((max_ch_disp-1) * offset))
    
# if time.max() > max_t_disp:
#     range_opts.append(xrange_hook)

# if len(channel_curves) > max_ch_disp:
#     range_opts.append(yrange_hook)

eeg_viewer = hv.Overlay(channel_curves, kdims="Channel").opts(
    width=800, height=600, padding=0, xlabel="Time (s)", ylabel="Channel",
    yticks=yticks, show_legend=False,) #hooks=range_opts,

y_positions, _ = zip(*yticks)
z_data = zscore(data, axis=1)
minimap = hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)")
minimap = minimap.opts(
    cmap="RdBu_r", colorbar=False, width=800, height=100, xlabel='', alpha=.5, yticks=[yticks[0], yticks[-1]],)

# Create RangeToolLink between the minimap and the main EEG viewer 
# (quirk: apply to just one eeg trace and it will apply to all. see HoloViews #4472)
RangeToolLink(minimap, list(eeg_viewer.values())[0], axes=["x", "y"], boundsx=(None, max_t_disp), boundsy=(None, np.max(data[max_ch_disp-1,:] + ((max_ch_disp-1) * offset))))

# Display vertically
layout = (eeg_viewer + minimap).cols(1)
layout.opts(shared_axes=False, merge_tools=False)



<div class="alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Tip:</p>
Hover near any border of the minimap, then click and drag to adjust the visible bounds in the EEG plot

</div>

## Load and plot real data